In [2]:
# Import Dependencies
import pandas as pd
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import os

## Create a list of Hospital URLS for hospitals in United States from American Hospital Directory

In [4]:
## URL for search of just US States. Returns 6962 Hostpitals
#### https://www.ahd.com/list_cms.php?submitted=Submit&mname=&mcity=&mstate%5B%5D=AK&mstate%5B%5D=AL&mstate%5B%5D=AR&mstate%5B%5D=AS&mstate%5B%5D=AZ&mstate%5B%5D=CA&mstate%5B%5D=CO&mstate%5B%5D=CT&mstate%5B%5D=DC&mstate%5B%5D=DE&mstate%5B%5D=FL&mstate%5B%5D=GA&mstate%5B%5D=HI&mstate%5B%5D=IA&mstate%5B%5D=ID&mstate%5B%5D=IL&mstate%5B%5D=IN&mstate%5B%5D=KS&mstate%5B%5D=KY&mstate%5B%5D=LA&mstate%5B%5D=MA&mstate%5B%5D=MD&mstate%5B%5D=ME&mstate%5B%5D=MI&mstate%5B%5D=MN&mstate%5B%5D=MO&mstate%5B%5D=MS&mstate%5B%5D=MT&mstate%5B%5D=NC&mstate%5B%5D=ND&mstate%5B%5D=NE&mstate%5B%5D=NH&mstate%5B%5D=NJ&mstate%5B%5D=NM&mstate%5B%5D=NV&mstate%5B%5D=NY&mstate%5B%5D=OH&mstate%5B%5D=OK&mstate%5B%5D=OR&mstate%5B%5D=PA&mstate%5B%5D=RI&mstate%5B%5D=SC&mstate%5B%5D=SD&mstate%5B%5D=TN&mstate%5B%5D=TX&mstate%5B%5D=UT&mstate%5B%5D=VA&mstate%5B%5D=VT&mstate%5B%5D=WA&mstate%5B%5D=WI&mstate%5B%5D=WV&mstate%5B%5D=WY&mzip=&mphone=


urllist = "https://www.ahd.com/list_cms.php?submitted=Submit&mname=&mcity=&mstate%5B%5D=AK&mstate%5B%5D=AL&mstate%5B%5D=AR&mstate%5B%5D=AS&mstate%5B%5D=AZ&mstate%5B%5D=CA&mstate%5B%5D=CO&mstate%5B%5D=CT&mstate%5B%5D=DC&mstate%5B%5D=DE&mstate%5B%5D=FL&mstate%5B%5D=GA&mstate%5B%5D=HI&mstate%5B%5D=IA&mstate%5B%5D=ID&mstate%5B%5D=IL&mstate%5B%5D=IN&mstate%5B%5D=KS&mstate%5B%5D=KY&mstate%5B%5D=LA&mstate%5B%5D=MA&mstate%5B%5D=MD&mstate%5B%5D=ME&mstate%5B%5D=MI&mstate%5B%5D=MN&mstate%5B%5D=MO&mstate%5B%5D=MS&mstate%5B%5D=MT&mstate%5B%5D=NC&mstate%5B%5D=ND&mstate%5B%5D=NE&mstate%5B%5D=NH&mstate%5B%5D=NJ&mstate%5B%5D=NM&mstate%5B%5D=NV&mstate%5B%5D=NY&mstate%5B%5D=OH&mstate%5B%5D=OK&mstate%5B%5D=OR&mstate%5B%5D=PA&mstate%5B%5D=RI&mstate%5B%5D=SC&mstate%5B%5D=SD&mstate%5B%5D=TN&mstate%5B%5D=TX&mstate%5B%5D=UT&mstate%5B%5D=VA&mstate%5B%5D=VT&mstate%5B%5D=WA&mstate%5B%5D=WI&mstate%5B%5D=WV&mstate%5B%5D=WY&mzip=&mphone="
urlsite = urlopen(urllist)
urlsoup = BeautifulSoup(urlsite.read(), "html.parser")

In [5]:
site = urlsoup.findAll("tr", {"class":{"b", ""}}) 
    
listofhosp = []


for hosp in site:
    if 'href'[1] in hosp.attrs:
        listofhosp.append("https://www.ahd.com"+current.attrs['href'])

In [7]:
df = pd.DataFrame(listofhosp, columns=["Links"])
df.to_csv('hospitallinks.csv')

### This function scrapes the tables on the hospital profile page on the AHD website and saves as csv files.  This information will be used in cluster analysis for hospitals.

In [6]:
def getprofile(hospitallink):
    
    hospital = hospitallink
    # We will want the above to eventually be a list of all US Hospital URLs.

    site = urlopen(hospital)
    soup = BeautifulSoup(site.read(), "html.parser")

    ### Regular expression to extract the 6 digit CMS code from the URL (used to name the csvs later)
    match = re.search(r'(?<!\d)\d{6}(?!\d)', hospital)
    csvname = (match.group(0))
    
    
    ### This scrapes Identification and characteristics table 

    rows = soup.findAll("tr")

    lists = []
    for x in rows:
        cells = x.findAll("td")
        lis = []
        for j in cells:
            Q = j.get_text()
            str_cells = str(Q)
            clean = re.compile("<.*?>")
            clean2 = (re.sub(clean, "",str_cells))
            lis.append(clean2.replace("\n", " "))
        lists.append(lis)


    lists

    K = lists[4]
    R = pd.DataFrame(K)
    R
    S = R.drop(R.index[[0,9,10,17,18, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]])
    S


    T = S.iloc[::2]
    T1 = T.reset_index().drop(["index"], axis = 1)
    T1.rename(columns = {0: "Attributes"}, inplace = True)
    T1


    U = S.iloc[1::2]
    U1 = U.reset_index().drop(["index"], axis = 1)
    U1


    T1['Details'] = U1[0]
    
    T1.to_csv(csvname+"_HospChar.csv")
    
    #Inpatient Utilization Table SCRAPES ALL NEEDED INFORMATION

    inpatientutil = soup.find("a", {"name":"inpatientstats"})
    inpatstats = inpatientutil.find_next("table")

    headerlist = ['', 'Number Medicare Inpatients', 'Average Length of Stay', 'Average Charges', 'Medicare Case Mix Index (CMI)']
    deptlist = []
    numpats = []
    avgstay = []
    avgcharge = []
    cmiindex = []

    #For loop starts at 1 because first row is headers which is different from other trs so it won't work.
    for statistics in inpatstats.findAll("tr")[1:]:
        data = statistics.findAll("td")
        deptlist.append(data[0].text)
        numpats.append(data[1].text)
        avgstay.append(data[2].text)
        avgcharge.append(data[3].text)
        cmiindex.append(data[4].text)
        
    ####Inputting inpatient utilization into a dataframe.

    InpatData = pd.DataFrame({headerlist[0]:deptlist, headerlist[1]:numpats, headerlist[2]:avgstay,
                             headerlist[3]:avgcharge, headerlist[4]:cmiindex})

    InpatData.to_csv(csvname+"_InpatData.csv", index = False)
    
    
    ###Inpatient origin for top 3 ZIP Codes

    inpatientorigin = soup.find("a", {"name":"inpatientorigin"})
    originstats = inpatientorigin.find_next("table")

    originheaders = ['ZIP Code of Residence', 'Discharges', 'Days of Care', 'Charges', 'Discharges Inc/(Dec)', 'Market Share']
    zipres = []
    dischargelist = []
    numberdays = []
    chgs = []
    changedischg = []
    mktshare = []

    #For loop starts at 1 because first row is headers which is different from other trs so it won't work.
    for ostats in originstats.findAll("tr")[1:]:
        origindata = ostats.findAll("td")
        zipres.append(origindata[0].text)
        dischargelist.append(origindata[1].text)
        numberdays.append(origindata[2].text)
        chgs.append(origindata[3].text)
        changedischg.append(origindata[4].text)
        mktshare.append(origindata[5].text)
        
        
    ####Inputting origin for top 3 zips into a dataframe. 

    OriginPatData = pd.DataFrame({originheaders[0]:zipres, originheaders[1]:dischargelist, originheaders[2]:numberdays,
                                 originheaders[3]:chgs, originheaders[4]:changedischg, originheaders[5]:mktshare})
    
    OriginPatData.to_csv(csvname+"_IPOrigin.csv", index = False)
    
    
    ### Scraping outpatient utilization.

    OPheader = soup.find("a", {"name":"outpatientstats"})
    OPstats = OPheader.find_next("table")

    opheads = ['APC Number', 'APC Description', 'Number Patient Claims', 'Average Charge', 'Average Cost']
    apcnum = []
    apcdesc = []
    numclaims = []
    opcharge = []
    opcost = []

    #For loop starts at 1 because first row is headers which is different from other trs so it won't work.
    for op in OPstats.findAll("tr")[1:]:
        opdata = op.findAll("td")
        apcnum.append(opdata[0].text)
        apcdesc.append(opdata[1].text)
        numclaims.append(opdata[2].text)
        opcharge.append(opdata[3].text)
        opcost.append(opdata[4].text)
        
        
    ####Inputting outpatient utilization into a dataframe. 

    OPUtilData = pd.DataFrame({opheads[0]:apcnum, opheads[1]:apcdesc, opheads[2]:numclaims,
                                 opheads[3]:opcharge, opheads[4]:opcost})
    
    
    OPUtilData.to_csv(csvname+"_OpUtilData.csv", index = False)
    
    
    
    ## Beds and Patient Days by Unit

    bedsheader = soup.find("a", {"name":"bedsdaysbyunit"})
    bedsstats = bedsheader.find_next("table")

    bedheads = [' ', 'Available Beds', 'Inpatient Days']
    unitlist = []
    availbeds = []
    inpatdays = []

    #For loop starts at 1 because first two rows is headers which is different from other trs so it won't work.
    for bed in bedsstats.findAll("tr")[2:]:
        beddata = bed.findAll("td")
        unitlist.append(beddata[0].text)
        availbeds.append(beddata[1].text)
        inpatdays.append(beddata[2].text)
        
    ####Inputting beds/patient days into a dataframe. 
   

    BedDaysData = pd.DataFrame({bedheads[0]:unitlist, bedheads[1]:availbeds, bedheads[2]:inpatdays})

    BedDaysData.to_csv(csvname+"_BedDaysData.csv", index = False)
    
    
    #Financial Statistics

    finheader = soup.find("a", {"name":"financialstats"})
    finstats = finheader.find_next("table")

    finheads = [' ', 'Available Beds', 'Inpatient Days']
    revtypes = []
    dollars = []
    percentages = []

    #For loop starts at 1 because first two rows is headers which is different from other trs so it won't work.
    for fins in finstats.findAll("tr")[1:]:
        findata = fins.findAll("td")
        revtypes.append(findata[0].text)
        dollars.append(findata[1].text)
        percentages.append(findata[2].text)
        
    ####Inputting financial stats into a dataframe. LOOKS GOOD

    FinData = pd.DataFrame({finheads[0]:revtypes, finheads[1]:dollars, finheads[2]:percentages})
    
    #Writing FinData to csv.
    FinData.to_csv(csvname+"_FinData.csv", index = False)
    
    print(FinData)
    print(T1)
    
    return

## Iterate through the links 3 hospitals at a time (more than three at a time will lead to errors due to captchas)

In [ ]:
# hospitallinks = pd.read_csv("D:/My Passport/Courses/UNH/Fall 2018/Health Analytics 812/links.csv")

# hospitallinks

for i in range(27, 29):
    getprofile(hospitallinks["Links"][i])